In [ ]:
import os
import torch

from utils.dataloader import get_dataloaders
from utils.train import train_model, test_model

from utils.metrics import (
    compute_full_metrics,
    save_results_csv,
    plot_confusion_matrix,
    plot_roc_auc,
    save_grayscale_samples,
    save_epoch_history_excel
)

from models.single_models import get_single_model
from utils.gradcam import GradCAM, save_gradcam_samples

import gc

C:\Users\navne\anaconda3\envs\lung_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Config

In [ ]:
DATA_DIR = "lung_ct_split"
BATCH_SIZE = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

EPOCH_OPTIONS = [20,25,30]

MODELS = [
    "resnet50",
    # "vgg16",
    # "densenet121",
    # "b4",
    # "efficientnet",
    # "inception",
    # "vit",
]

PREPROCESSING_CONFIGS = {
    "baseline": {},
    # "clahe": {"clahe": True},
    # "clahe_median": {"clahe": True, "median": True},
}


CSV_PATH = "results/reports/final_results.csv"


# Ensure Folders 

In [4]:
os.makedirs("results/roc_auc", exist_ok=True)
os.makedirs("results/confusion", exist_ok=True)
os.makedirs("results/grayscale", exist_ok=True)
os.makedirs("saved_models", exist_ok=True)

# Experiment Loop

In [ ]:
for epochs in EPOCH_OPTIONS:
    for model_name in MODELS:
        for prep_name, preprocess_cfg in PREPROCESSING_CONFIGS.items():

            exp_id = f"{model_name}_{prep_name}_ep{epochs}"
            print(f"\n\n🚀 RUNNING EXPERIMENT: {exp_id}")

            train_loader, val_loader, test_loader, class_names = get_dataloaders(
                data_dir=DATA_DIR,
                batch_size=BATCH_SIZE,
                preprocess_config=preprocess_cfg,
            )

            model = get_single_model(model_name, num_classes=len(class_names)).to(DEVICE)

            model, history = train_model(
                model,
                train_loader,
                val_loader,
                DEVICE,
                epochs=epochs,
                model_name=exp_id,
            )

            save_epoch_history_excel(history, model_name=preprocess_cfg,save_path=f"results/{exp_id}/expriment_logs.xlsx")

            test_acc, report, cm, labels, preds, probs, images = test_model(
                model,
                test_loader,
                DEVICE,
                class_names,
                return_details=True,
            )

            print(f"\n🔥 Test Accuracy: {test_acc:.4f}")
            print(report)

            accuracy, precision, recall, f1 = compute_full_metrics(labels, preds)

            roc_path = f"results/roc_auc/{exp_id}.png"
            auc_score = plot_roc_auc(labels, probs, class_names, roc_path)

            cm_path = f"results/confusion/{exp_id}.png"
            plot_confusion_matrix(cm, class_names, cm_path)

            gray_dir = f"results/grayscale/{exp_id}"
            save_grayscale_samples(images, gray_dir)

            target_layer = None

            try:
                backbone = model[0]  # because model is Sequential(backbone, classifier)
            except:
                backbone = model     # for VGG direct return case
            
            if "resnet" in model_name:
                target_layer = backbone.layer4[-1]
            
            elif "efficientnet" in model_name:
                target_layer = backbone.features[-1]
            
            elif "densenet" in model_name:
                target_layer = backbone.features[-1]
            
            elif "vgg" in model_name:
                target_layer = backbone.features[-1]
            else:
                target_layer = None

            if target_layer is not None:
                gradcam = GradCAM(model, target_layer)
                
                save_gradcam_samples(
                    gradcam,
                    images,
                    class_names,
                    save_dir=f"results/gradcam/{exp_id}",
                    device=DEVICE,
                )

            save_results_csv(
                {exp_id: accuracy},
                CSV_PATH,
                preprocess_cfg=preprocess_cfg,
                epochs=epochs,
                precision=precision,
                recall=recall,
                f1=f1,
                auc_score=auc_score,
            )

            del model
            del train_loader, val_loader, test_loader
            torch.cuda.empty_cache()
            gc.collect()
print("\n\n🏁 ALL EXPERIMENTS COMPLETED SUCCESSFULLY")



🚀 RUNNING EXPERIMENT: resnet50_baseline_ep5

Epoch 1/5
Train Loss: 0.8330 | Train Acc: 0.6128
Val   Loss: 0.7443 | Val   Acc: 0.7988

Epoch 2/5
Train Loss: 0.5338 | Train Acc: 0.7966
Val   Loss: 0.4856 | Val   Acc: 0.8110

Epoch 3/5
Train Loss: 0.3896 | Train Acc: 0.8579
Val   Loss: 0.3511 | Val   Acc: 0.8720
